# Big O notation

Big O notation is a way of catagorizing algorithms that only pays attention to the way cost grows as we process more data.

What is the big O of the following?

1. Find an item in a list.
2. Find an item in a dictionary.
3. Sorting a list by finding the smallest item, then the next smallest item, continuing for each element in turn.
4. Sort a list by putting all elements in random order until they happen to be in sorted order.

Given an array of integers, find the maximum sum of any contiguous slice. If the maximum is negative, return zero.

```
-------------------------------------------------------
| 31 | -41 | 59 | 26 | -53 | 58 | 97 | -93 | -23 | 84 |
-------------------------------------------------------
           ↑                    ↑
           2                    6
```

Here the slice `[2:6]` a sum of 187, which is the maximum of all possible slices `[i:j]`.

What is the worst, simpelest strategy we could use to find the maximum sum?

In [26]:
import numpy

values = numpy.array([31, -41, 59, 26, -53, 58, 97, -93, -23, 84])

def solution_1(values):
    current_max = 0
    for i in range(len(values)):
        for j in range(i, len(values)):
            current_max = max(current_max, sum(values[i:j]))
            
    return current_max
        
print(solution_1(values))

187


What is the performance of this algorithm?
1. $O(n)$
2. $O(n log(n))$
3. $O(n ^ 2)$
4. $O(n ^ 3)$

# Duplicate work

The inner loop adds the same numbers together over and over.  There are two options to avoid this.

1. Accumulate the sum as we iterate, instead of computing it again each time.

2. Calculate the sums once ahead of time, and look them up later.

For the second option, the trick is that
```python
sum(array[i:j]) = sum(array[:j]) - sum(array[:i-1])
```
and that we can compute the cumulative sum of the array ahead of time.

Try adapting your implementation with one of these strategies.

In [22]:
def solution_2(values):
    current_max = 0
    for i in range(len(values)):
        current_sum = 0
        for val in values[i:]:
            current_sum += val
            current_max = max(current_max, current_sum)
    
    return current_max
            
print(solution_2(values))

187


In [25]:
def solution_3(values):
    cumulative = numpy.zeros(len(values) + 1, dtype=int)
    current_sum = 0

    # This is equivalent to itertools.accumulate plus a leading zero
    for i, v in enumerate(values):
        current_sum += v
        cumulative[i + 1] = current_sum
    
    current_max = 0
    for i, cumulative_i in enumerate(cumulative):
        for j, cumulative_j in enumerate(cumulative[i:]):
            current_sum = cumulative_j - cumulative_i
            current_max = max(current_max, current_sum)
    
    return current_max

print(solution_3(values))


187


# Divide and Conquor

There is a tricky way of going faster, by noticing that we can divide this problem in two.

```
          left maximum         right maximum
-----------===========------    ===========-----------------
| 31 | -41 | 59 | 26 | -53 |    | 58 | 97 | -93 | -23 | 84 |
-----------===========-------   ===========-----------------
```

We can keep doing this recursively

```
       ------===========------         At each node we have three options:
       | -41 | 59 | 26 | -53 |           1. left child's max
       ------===========------           2. right child's max
            /           \                3. maximum crosses the middle
           /             \           
     ------======    ======------    
     | -41 | 59 |    | 26 | -53 |    
     ------======    ======------    
      /      \          /     \      
-------    ======    =====-    -------
| -41 |    | 59 |    | 26 |    | -53 |
-------    ======    =====-    -------
```



Checking the third case takes O(n) at each node.
- What is the big O complexity of this algorithm?
- If we didn't have to worry about the third case, what would be the complexity?

# Dynamic programming

Let's say we have a solution to a list with N elements.  If we add an element on to the end, how do we need to modify the solution?

```
======-----------       ------
| 59 | -41 | 31 |   +   | 30 |    =    ?
======-----------       ------
```

In [47]:
def solution_4(values):
    current_max = 0
    max_ending_here = 0
    
    for val in values:
        max_ending_here = max(max_ending_here + val, 0)
        current_max = max(current_max, max_ending_here)
        
    return current_max

print(solution_4(values))

187


Rules to remember:
- Don't do work you don't have to do
- Look it up instead of finding it each time
- Look for ways to divide the problem into subproblems that can be combined into the full solution.
- Look for simpler subproblems that can be extended into a full solution.